### Разминка

Рассказать про материал второго курса, домашние задания и систему выставления оценок.

План на лекцию: освежить в памяти основы из 1-й части курса.

<br />

**Разминка 1**: обсудить код программы по выводу top-n слов песни "Yellow Submarine"

```c++
// обсудить: что это и как оно работает
#include <algorithm>
#include <cctype>
#include <exception>
#include <iostream>
#include <iterator>
#include <sstream>
#include <string>
#include <tuple>
#include <unordered_map>
#include <utility>
#include <vector>

// обсудить:
//   * что это
//   * что значит const
//   * что значит static
//   * где живут данные
//   * когда объект создаётся и когда уничтожается
static const std::string song = "\
    Yellow Submarine\n\
    \n\
    In the town where I was born\n\
    Lived a man who sailed to sea\n\
    And he told us of his life\n\
    In the land of submarines\n\
    So we sailed up to the sun\n\
    Till we found a sea of green\n\
    And we lived beneath the waves\n\
    in our yellow submarine\n\
    \n\
    We all live in a yellow submarine\n\
    Yellow submarine, yellow submarine\n\
    We all live in a yellow submarine\n\
    Yellow submarine, yellow submarine\n\
    \n\
    And our friends are all aboard\n\
    Many more of them live next door\n\
    And the band begins to play\n\
    \n\
    We all live in a yellow submarine\n\
    Yellow submarine, yellow submarine\n\
    We all live in a yellow submarine\n\
    Yellow submarine, yellow submarine\n\
    \n\
    (Full speed ahead Mr. Parker, full speed ahead\n\
    Full speed ahead it is, Sergeant\n\
    Action station, action station\n\
    Aye, aye, sir, fire\n\
    Captain, captain)\n\
    \n\
    As we live a life of ease\n\
    Every one of us has all we need\n\
    Sky of blue and sea of green\n\
    In our yellow submarine\n\
    \n\
    We all live in a yellow submarine\n\
    Yellow submarine, yellow submarine\n\
    We all live in a yellow submarine\n\
    Yellow submarine, yellow submarine\n\
    We all live in a yellow submarine\n\
    Yellow submarine, yellow submarine";

// обсудить:
//   * что это
//   * какой код ассемблера генерирует эта строчка
//   * что такое unordered_map, внутреннее устройство, разница с std::map
using WordsCounter = std::unordered_map<std::string, int>;

// обсудить:
//   * что значит static
//   * что значит &
static void to_lower_inplace(std::string& s)
{
    for (char& c : s)
        c = static_cast<char>(std::tolower(static_cast<unsigned char>(c)));
}

// обсудить:
//   * зачем erase
static void remove_non_alpha_inplace(std::string& s)
{
    s.erase(
        std::remove_if(
            s.begin(),
            s.end(),
            [](unsigned char c) { return !std::isalpha(c); }),
        s.end());
}

// обсудить:
//   * что значит nodiscard
//   * почему const&? правила передачи аргументов
//   * что даёт наличие return только от одной локальной переменной в функции
//   * как устроен в памяти std::vector<std::string>. если сложно - рисовать
//   * какие ещё есть контейнеры-последовательности кроме std::vector, как они устроены
[[nodiscard]] static std::vector<std::string> split_by_words(const std::string& text)
{
    std::vector<std::string> words;

    std::istringstream ss{text};
    std::for_each(
        std::istream_iterator<std::string>(ss),
        std::istream_iterator<std::string>(),
        [&](std::string s){
            remove_non_alpha_inplace(s);
            to_lower_inplace(s);
            if (!s.empty())
                words.emplace_back(std::move(s));
        });

    return words;
}

[[nodiscard]] static WordsCounter make_words_counter(const std::string& text)
{
    const std::vector<std::string> words = split_by_words(text);

    WordsCounter counter;
    for (const std::string& word : words)
        counter[word] += 1;
    
    return counter;
}

// обсудить:
//   * почему namespace
//   * почему struct
//   * разница class/struct
namespace {
struct CountAndWord
{
    int count;
    std::string word;
};
}  // namespace

// обсудить:
//   * reserve
//   * partial_sort (+begin/end)
//   * lambda (+captures)
static void print_top_n_words(const WordsCounter& counter, const int topn)
{
    if (topn <= 0)
        return;

    std::vector<CountAndWord> caws;
    caws.reserve(counter.size());
    for (const auto& [word, count] : counter)
        caws.emplace_back(CountAndWord{ count, word });

    const int top_ix = std::min<int>(topn, caws.size());

    std::partial_sort(
        caws.begin(),
        caws.begin() + top_ix,
        caws.end(),
        [](const CountAndWord& l, const CountAndWord& r) {
            return std::tie(r.count, r.word) < std::tie(l.count, l.word);
        });

    for (int i = 0; i < top_ix; ++i)
        std::cout << caws[i].word << " -> " << caws[i].count << '\n';
}

// обсудить:
//   * что такое main
//   * что такое argc, argv
int main(int argc, char **argv)
{
    // обсудить:
    //   * почему 2
    //   * что в argv[0]
    //   * что такое endl
    //   * почему return 1
    if (argc != 2)
    {
        std::cout << "Usage: " << argv[0] << " top_n" << std::endl;
        return 1;
    }

    // обсудить:
    //   * что такое try-catch    
    try
    {
        const int top_n = std::stoi(argv[1]);

        const WordsCounter words_counter = make_words_counter(song);

        print_top_n_words(words_counter, top_n);
    }
    catch (const std::exception& e)
    {
        // обсудить:
        //   * что такое cerr
        //   * почему return 1
        //   * что такое std::exception и почему он здесь
        std::cerr << "ERROR: failed to find top n words: " << e.what() << std::endl;
        return 1;
    }
    
    // просмотреть всю программу, где могут быть брошены исключения
    return 0;
}
```

<br />

**Разминка 2**: обсудить класс `RoundRobinQueue`

```c++
#include <array>
#include <initializer_list>
#include <iostream>
#include <optional>
#include <stdexcept>
#include <utility>

// обсудить:
//   * шаблоны (что такое, зачем, как)
//   * шаблонные параметры
template<typename T, int N>
class RoundRobinQueue
{
private:
    // обсудить: инварианты
    std::array<std::optional<T>, N> data;
    int start_ix = 0;  // индекс первого элемента в очереди
    int final_ix = 0;  // индекс следующего за последним элементом в очереди
    
public:
    // обсудить:
    //   * конструкторы, деструкторы, когда вызываются
    //   * какие есть ещё спец. методы
    //   * какие есть правила
    
    RoundRobinQueue() = default;

    RoundRobinQueue(std::initializer_list<T> lst)
    {
        for (const T& t : lst)
            push(t);
    }

    RoundRobinQueue(const RoundRobinQueue&) = default;
    RoundRobinQueue& operator=(const RoundRobinQueue&) = default;

    // обсудить:
    //   * что это такое
    //   * что делает std::move
    //   * зачем присваивать rhs
    //   * зачем noexcept
    RoundRobinQueue(RoundRobinQueue&& rhs) noexcept
        : data(std::move(rhs.data))
        , start_ix(rhs.start_ix)
        , final_ix(rhs.final_ix)
    {
        rhs = RoundRobinQueue();
    }

    // обсудить:
    //   * зачем это нужно если есть move-конструктор
    //   * зачем проверка на this
    RoundRobinQueue& operator=(RoundRobinQueue&& rhs) noexcept
    {
        if (this != &rhs)
        {
            data = std::move(rhs);
            start_ix = rhs.start_ix;
            final_ix = rhs.final_ix;

            rhs = RoundRobinQueue();
        }
        return *this;
    }

    // обсудить:
    //   * что это и что здесь происходит
    ~RoundRobinQueue() = default;

    // обсудить: почему const
    bool empty() const
    {
        return start_ix == final_ix && !data[start_ix].has_value();
    }

    bool full() const
    {
        return start_ix == final_ix && data[start_ix].has_value();
    }

    T pop()
    {
        if (empty())
            throw std::runtime_error("pop from empty queue");

        T res = std::move(data[start_ix].value());
        data[start_ix].reset();
        start_ix = next_ix(start_ix);
        return res;
    }

    void push(T item)
    {
        if (full())
            throw std::runtime_error("push to full queue");

        data[final_ix].emplace(std::move(item));
        final_ix = next_ix(final_ix);
    }

private:
    // обсудить:
    //   * что значит static
    static int next_ix(const int ix)
    {
        return (ix + 1) % N;
    }
};

int main()
{
    try
    {
        RoundRobinQueue<std::string, 3> q;
        q.push("alesha");
        q.push("dobrynia");
        q.push("ilya");

        while (true)
            std::cout << q.pop() << std::endl;
    }
    catch (const std::exception& e)
    {
        std::cout << e.what() << std::endl;
    }
    
    // обсудить:
    //   * что будет выведено
    //   * будет ли скомпилирован конструктор RoundRobinQueue(std::initializer_list<T> lst)
    //   * сколько классов RoundRobinQueue будет скомпилировано
    //   * как будут линковаться несколько RoundRobinQueue<std::string, 3>, если они компилируются
    //     в разных cpp-файлах
    return 0;
}
```

<br />

**Разминка 3**: обсудить программу, печатающую животных в зоопарке

```c++
#include <cstdio>
#include <exception>
#include <iostream>
#include <memory>
#include <string>
#include <utility>
#include <vector>

using namespace std;

namespace {

// замечание:
//
//   организуем животных в иерархию:
//
//   Animal
//    |
//   Turtle
//    |
//   NinjaTurtle
    
class Animal
{   
public:
    Animal(const string& name, int age)
        : name_(name)
        , age_(age)
    {}

    // обсудить:
    //   * что это такое
    //   * зачем так делают
    virtual ~Animal() = default;

    // обсудить:
    //   * что это такое
    //   * = 0
    virtual string greeting() const = 0;

    const string& name() const { return name_; }
    int age() const { return age_;  }
    
private:
    string name_;
    int age_;
};

class Turtle : public Animal
{
public:
    Turtle(const string& name, int age)
        : Animal(name, age)
    {}

    // обсудить:
    //   * override
    //   * final
    string greeting() const override
    {
        return "hello";
    }
};

// обсудить:
//   * порядок вызова конструкторов-деструкторов
//   * layout класса (не трогаем что это non-standard layout)
//   * sizeof класса
//   * alignment
class NinjaTurtle : public Turtle
{
public:
    NinjaTurtle(const string& name,
                const string& short_name)
        : Turtle(name, 12)
        , short_name_(short_name)
    {}

    string greeting() const override
    {
        return "camabanga!";
    }
    
private:
    string short_name_;
};

}  // namespace

// обсудить:
//   * что такое unique_ptr и что вы про него знаете
//   * какие ещё есть умные указатели, внутреннее устройство
//   * зачем нужен make_unique
//   * зачем нужен make_shared
[[nodiscard]]
static vector<unique_ptr<Animal>> make_zoo()
{
    vector<unique_ptr<Animal>> rv;
    rv.reserve(7);
    rv.emplace_back(make_unique<Turtle>("Tortilla", 100));
    rv.emplace_back(make_unique<Turtle>("Big Turtle", 100));
    rv.emplace_back(make_unique<Turtle>("Aunt Motya", 200));
    rv.emplace_back(make_unique<NinjaTurtle>("Donatello"));
    rv.emplace_back(make_unique<NinjaTurtle>("Rafael"));
    return move(rv);
}

// обсудить:
//   * где в коде main происходит вызов виртуальных функций (подсказка: 2 места)
//
int main()
{
    for (const unique_ptr<Animal>& a : make_zoo())
    {
        printf("I'm %10s. My age is %3i. %s\n",
               a->name().c_str(), a->age(), a->greeting().c_str());
    }
    return 0;
}
```

<br />

##### Forward declaration: традиционный пример с iostream

Рассмотрим компиляцию cpp-файла:

```c++
// example_fwd_declaration_1.cpp

#include <iostream>

extern int read_int(std::istream& is);
extern int read_float(std::istream& is);

struct Person
{
    int age;
    float weight;
};

Person read_person(std::istream& is)
{
    Person p;
    p.age = read_int(is);
    p.weight = read_float(is);
    return p;
}
```

In [12]:
# скомпилируем, замерим время
!time --format "real time %E\nuser time %U\nsys  time %S" clang++-8 -c example_fwd_declaration_1.cpp

real time 0:00.24
user time 0.20
sys  time 0.02


<br />

Заметим, что при компиляции `example_fwd_declaration_1.cpp` компилятору нет необходимости видеть реализацию класса `std::istream`, т.к. в рамках этого файла:
* ни один из методов класса не вызывается
* не нужно знать размер класса, т.к. передаётся только указатель на объект, а размер указателя известен

_(пройтись ещё раз по коду, показать где и как используется класс `std::istream`)_

Компилятору достаточно знать, что такой класс существует.

Применим приём **forward declaration** - заменим definition класса на declaration.

Класс `std::istream` - шаблонный, поэтому forward declaration будет выглядеть страшненько:

```c++
// forward-declare std::istream
namespace std {
  template<class CharT> struct char_traits;
  template<class CharT, class Traits = char_traits<CharT>> class basic_istream;
  using istream = basic_istream<char>;
}

extern int read_int(std::istream& is);
extern int read_float(std::istream& is);

struct Person
{
    int age;
    float weight;
};

Person read_person(std::istream& is)
{
    Person p;
    p.age = read_int(is);
    p.weight = read_float(is);
    return p;
}
```

In [20]:
# скомпилируем, замерим время
!time --format "real time %E\nuser time %U\nsys  time %S" clang++-8 -c example_fwd_declaration_2.cpp

real time 0:00.02
user time 0.01
sys  time 0.00


**Мы ускорили компиляцию в ~10 раз**

_(объяснить, почему возникает такой эффект)_

<br />

##### Forward declaration: iosfwd

Т.к. проблема времени компиляции для заголовочного файла `iostream` известна, для него давно в стандарт ввели `#include <iosfwd>` - forward declaration для объектов из `iostream`

https://en.cppreference.com/w/cpp/header/iosfwd

Поэтому наш пример можно упростить:

```c++
#include <iosfwd>

extern int read_int(std::istream& is);
extern int read_float(std::istream& is);

struct Person
{
    int age;
    float weight;
};

Person read_person(std::istream& is)
{
    Person p;
    p.age = read_int(is);
    p.weight = read_float(is);
    return p;
}
```

In [24]:
# скомпилируем, замерим время
!time --format "real time %E\nuser time %U\nsys  time %S" clang++-8 -c example_fwd_declaration_3.cpp

real time 0:00.03
user time 0.02
sys  time 0.00


Здесь можно найти ориентировочное влияние include-а на время компиляции:<br />
https://artificial-mind.net/projects/compile-health/

<br />

##### Forward declaration: стандартный пример

Вспомним пример из домашнего задания про фигурки 1-го семестра.

Вариант организации запуска интерпретатора:

```c++
//
// Interpret.h
//
#include "Scene.h"
#include <iostream>

void run_interpreter(std::istream& is, Scene& scene);

//
// Interpret.cpp
//
#include "Interpret.h"

void run_interpreter(std::istream& is, Scene& scene) {
    /* impl */
}
```

Вариант с применением forward declaration

```c++
//
// Interpret.h
//
#include <iosfwd>

class Scene;

void run_interpreter(std::istream& is, Scene& scene);

//
// Interpret.cpp
//
#include "Interpret.h"

#include "Scene.h"
#include <iostream>

void run_interpreter(std::istream& is, Scene& scene) {
    /* impl */
}
```

Но...:
* прописывать в каждом заголовочном файле `class Scene;` - неудобно
* ещё более неудобно, когда `class` со временем рефакторится в `struct`
* ещё более неудобно с шаблонами (записи длиннее, меняются чаще)

Решение... аналогично `iosfwd`:
* сгруппировать по смыслу несколько классов
* организовать для них общий header с forward declarations

Как это можно организовать на примере домашки с фигурками:

```c++
//
// InterpreterFwd.h - forward-declaration для классы сцены и фигурок
//
class Circle;
class Figure;
class Rectangle;
class Scene;
class Triangle;

//
// Interpret.h
//
#include "InterpreterFwd.h"
#include <iosfwd>

void run_interpreter(std::istream& is, Scene& scene);

//
// Interpret.cpp
//
#include "Interpret.h"

#include "Scene.h"
#include <iostream>

void run_interpreter(std::istream& is, Scene& scene) {
    /* impl */
}

//
// Scene.h
//
#include "InterpreterFwd.h"

class Scene {
    /* ... */
};
```

<br />

**Упражнение**: что в этом header можно сделать через forward declaration?

Обсудить всё:
* `std::string`
* `int`
* `std::ostream`
* `std::istream`
* `DBRecordPerson`
* `JsonValue`
* `Person`

```c++
#include <iostream>
#include <memory>
#include <string>

#include "Database/Records.h"
#incldue "Json/JsonValue.h"

struct Person
{
    std::string name;
    int age;    
};

inline
std::ostream& operator<<(std::ostream& os, const Person& p)
{
    return os << p.name << ', '<< p.age;
}

std::istream& operator>>(std::istream& is, Person& p);

std::unique_ptr<Person> create_person(const DBRecordPerson& record);
std::unique_ptr<Person> create_person(const JsonValue& json_obj);

JsonValue make_json_obj(const Person& p);
```

Что нужно поменять, чтобы `#include <iostream>` заменить на forward declaration?

<br />

##### Forward declaration: тонкости std::unique_ptr как поля класса

Вернёмся к домашней работе про фигурки. Рассмотрим пример такого заголовочного файла:

```c++
#include "Color.h"
#include "Figure.h"

struct SceneItem
{
    std::unique_ptr<Figure> geometry;
    int id;
    Color color;
};
```

**Вопрос:**
* как внутри устроен `std::unique_ptr`
* как сделать forward declaration для `Figure`

<details>
<summary>ответ</summary>
<p>
Не можем, потому что деструктор SceneItem автогенерируется в каждом файле, где используется, там автогенерируется деструктор std::unique_ptr, который обязан вызывать delete у Figure, а значит, он должен знать, деструктор Figure - виртуальный или нет.
</p> 
</details>

_(объяснить подробнее как работает ответ)_

<br />

**Вопрос:**
* как можно обойти проблему

Как тогда сделать forward declaration для `Figure`:

```c++
//
// SceneItem.h
//

#include "Color.h"

class Figure;

struct SceneItem
{
    // объявить деструктор и move-операции (где нужно делать delete)
    ~SceneItem();    
    SceneItem(SceneItem&&) noexcept;
    SceneItem& operator=(SceneItem&&) noexcept;
    
    // дефолтный конструктор (следуем правилу 6)
    SceneItem();

    // операции копирования можно удалить явно (следуем правилу 6)
    SceneItem(const SceneItem&) = delete;
    SceneItem& operator=(const SceneItem&) = delete;
    
    std::unique_ptr<Figure> geometry;
    int id;
    Color color;
};


//
// SceneItem.cpp
//
#include "SceneItem.h"

#include "Figure.h"

// скомилируется единожды (здесь нужен #include <Figure.h>)
SceneItem::~SceneItem() = default;

// скомилируется единожды (здесь нужен #include <Figure.h>)
SceneItem::SceneItem(SceneItem&&) noexcept = default;

// скомилируется единожды (здесь нужен #include <Figure.h>)
SceneItem& SceneItem::operator=(SceneItem&&) noexcept;
```

**Замечание:** Для `std::shared_ptr` в отличие от `std::unique_ptr` можно сделать forward declaration без отключения автогенерации деструктора. Почему? Почему так не делают?

<br />

##### отступление - паттерн pimpl

Паттрен pimpl (pointer to implementation) помогает минимизировать зависимости по include-ам в C++ проектах.

**Пример:**

```c++
//
// Network.h
//
#include <memory>
#include <string>

class Network
{
public:
    Network();
    ~Network();
    // + define move ops, delete copy ops
    
    std::string get(const std::string& url);

private:
    class Impl;
    std::unique_ptr<Impl> impl;
};


//
// Network.cpp
//
#include "Network.h"
#include "CoolNetworkLibrary/CoolNetworkLibrary.h"

class Network::Impl
{
public:
    std::string get(const std::string& url) {
        /* impl */
    }
    
private:
    CoolNetworkLibraryStaff staff;
};

Network::Network()
    : impl(std::make_unique<Impl>())
{}

Network::~Network() = default;

std::string Network::get(const std::string& url) {
    return impl->get(url);
}
```

**Замечание:**
* плюс - нет зависимости всего проекта по header-ам от `CoolNetworkLibrary`, зависимость локализована и не раздувает время компиляции
* плюс - стабильность ABI
* минус - запрещено копирование pimpl-объектов
* минус - подороже создание / удаление
* минус - подороже обращение к объекту

<br />

**Общая идея:** если хочется использовать возможности pimpl, делает это стоит с "большими" логическими объектами типа сеть / связь с БД / кеш запросов / парсер json-ов / ... - объекты, которые:
* создаются редко
* стоимость вызова достаточно велика по сравнению с одним прыжком по памяти
* не подразумевают копирования

<br />

##### Forward declaration: тонкости std::optional как поля класса

Заменим в примере про `SceneItem` `std::uinque_ptr` на `std::optional`:

```c++
#include <optional>
#include <string>

class Network
{
public:
    Network();
    ~Network();
    // + define move ops, delete copy ops

    std::string get(const std::string& url);

private:
    class Impl;
    std::optional<Impl> impl;
};
```

_(уточнить, что все понимают как устроен `std::optional` внутри)_

Таким образом экономим аллокацию, но подход не работает.

**Вопрос:** почему? 

Как заставить работать pimpl с `std::optional` - в докладе [Антона Полухина С++-трюки из Такси](https://www.youtube.com/watch?v=mkPTreWiglk)

<br />

##### Forward declaration: тонкости наследования

**Вопрос:** возможен ли forward declaration для такого cpp-файла и почему?

```c++
#include <iostream>

class Figure;
class Rectangle;

extern void draw_figure(const Figure& figure);

void process_rectangle(Rectangle& rect)
{
    std::cout << "drawing a rectangle figure" << std::endl;
    draw_figure(rect);  // <---
}
```

**Подсказка:** вспомните лекцию из первой части про множественное наследование и смещение базовых классов в наследнике

<details>
<summary>ответ</summary>
<p>Нет, т.к. не подглядев в реализацию, никак не узнать, что Rectangle - наследник от фигуры. А если он является наследником, то ещё нужно знать точное смещение базового класса Figure внутри Rectangle, т.к. возможен случай множественного наследования (или non-standard-layout, но про него говорить не будем)</p>
</details>

<br />

**Резюме:**
* Техника forward declaration позволяет ускорить компиляцию
* Forward declaration применим не всегда. FWD запрещён, если компилятору нужна информация о методах объекта, его размере или наследнике
* Идиома pimpl помогает снизить зависимости по заголовочным файлам, но не бесплатна в runtime